<a href="https://colab.research.google.com/github/Snowdenstyll/keno/blob/main/jupyterNotebooks/Keno_Lottery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [115]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [117]:
df = pd.read_csv("2024.csv")

In [118]:
df.head()
df.tail()
df.info()
df.describe()
df.drop(['PlayDate', 'AP'], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   PlayDate  130 non-null    object
 1   AP        130 non-null    object
 2   N01       130 non-null    int64 
 3   N02       130 non-null    int64 
 4   N03       130 non-null    int64 
 5   N04       130 non-null    int64 
 6   N05       130 non-null    int64 
 7   N06       130 non-null    int64 
 8   N07       130 non-null    int64 
 9   N08       130 non-null    int64 
 10  N09       130 non-null    int64 
 11  N10       130 non-null    int64 
 12  N11       130 non-null    int64 
 13  N12       130 non-null    int64 
 14  N13       130 non-null    int64 
 15  N14       130 non-null    int64 
 16  N15       130 non-null    int64 
 17  N16       130 non-null    int64 
 18  N17       130 non-null    int64 
 19  N18       130 non-null    int64 
 20  N19       130 non-null    int64 
 21  N20       130 no

In [119]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [120]:
# All our games
number_of_rows = df.values.shape[0]
# Amount of games we need to take into consideration for prediction
window_length = 10
number_of_features = df.values.shape[1]

X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)


In [121]:
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [122]:
# Recurrent Neural Netowrk (RNN) with Long Short Term Memory (LSTM)
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
batch_size = 90

# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(0.2))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(0.1))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a fourth Dropout layer
model.add(Dropout(0.2))
# Adding the first output layer
model.add(Dense(70))
# Adding the last output layer
model.add(Dense(number_of_features))

In [123]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.95, random_state=95)

# Train the model
history = model.fit(X, y, batch_size=batch_size, epochs=1000, verbose=2)
#history = model.fit(X_train, y_train, batch_size=100, epochs=600, verbose=2)

Epoch 1/1000
2/2 - 16s - loss: 1.0213 - accuracy: 0.0250 - 16s/epoch - 8s/step
Epoch 2/1000
2/2 - 0s - loss: 1.0154 - accuracy: 0.0417 - 51ms/epoch - 26ms/step
Epoch 3/1000
2/2 - 0s - loss: 1.0124 - accuracy: 0.0750 - 51ms/epoch - 25ms/step
Epoch 4/1000
2/2 - 0s - loss: 1.0085 - accuracy: 0.1167 - 52ms/epoch - 26ms/step
Epoch 5/1000
2/2 - 0s - loss: 1.0060 - accuracy: 0.1250 - 51ms/epoch - 26ms/step
Epoch 6/1000
2/2 - 0s - loss: 1.0019 - accuracy: 0.1167 - 53ms/epoch - 26ms/step
Epoch 7/1000
2/2 - 0s - loss: 0.9984 - accuracy: 0.1333 - 39ms/epoch - 19ms/step
Epoch 8/1000
2/2 - 0s - loss: 0.9941 - accuracy: 0.1167 - 38ms/epoch - 19ms/step
Epoch 9/1000
2/2 - 0s - loss: 0.9881 - accuracy: 0.1167 - 41ms/epoch - 21ms/step
Epoch 10/1000
2/2 - 0s - loss: 0.9864 - accuracy: 0.1167 - 37ms/epoch - 19ms/step
Epoch 11/1000
2/2 - 0s - loss: 0.9816 - accuracy: 0.1333 - 37ms/epoch - 19ms/step
Epoch 12/1000
2/2 - 0s - loss: 0.9772 - accuracy: 0.1167 - 38ms/epoch - 19ms/step
Epoch 13/1000
2/2 - 0s - lo

In [124]:
loss, accuracy = model.evaluate(X, y, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

4/4 [==============================] - 2s 10ms/step - loss: 0.0045 - accuracy: 0.9250
Test Loss: 0.0045404876582324505, Test Accuracy: 0.925000011920929
4/4 [==============================] - 0s 10ms/step - loss: 0.0045 - accuracy: 0.9211
Test Loss: 0.00450452184304595, Test Accuracy: 0.9210526347160339


In [110]:
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)

In [125]:
df1 = df.copy()
to_predict = df1.tail(window_length)
to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)
to_predict

<ipython-input-125-efb52fef0daf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)


,N01,N02,N03,N04,N05,N06,N07,N08,N09,N10,N11,N12,N13,N14,N15,N16,N17,N18,N19,N20
120,2,3,6,11,21,24,30,31,33,34,35,37,39,41,43,44,48,56,68,70
121,3,4,10,12,13,14,15,22,31,33,36,43,48,50,59,60,63,65,66,68
122,10,11,15,17,18,23,27,30,37,39,43,47,48,51,52,56,61,63,65,67
123,1,3,7,15,20,26,27,33,34,38,40,41,46,48,51,58,61,62,63,70
124,1,9,10,19,20,22,24,26,28,31,34,37,38,42,45,55,60,61,67,70
125,2,3,4,8,10,14,18,20,30,31,33,44,45,48,57,59,60,61,68,70
126,6,23,28,29,31,32,36,37,39,40,44,49,51,52,54,60,61,62,63,68
127,3,13,14,21,26,28,30,31,34,35,39,40,47,53,54,58,60,62,65,68
128,3,5,11,16,17,18,20,23,30,35,38,39,42,43,49,57,59,61,62,67


In [126]:
prediction = np.array(df.tail(1))

x_next = scaler.transform(to_predict)
y_pred = model.predict(np.array([x_next]))

temp = np.array(scaler.inverse_transform(y_pred).astype(int)[0])
combined_array = np.concatenate((temp, np.array(prediction[0])))
winning_numbers = np.bincount(combined_array)
winners = np.unique(combined_array)

uniques, uniq_idx, counts = np.unique(combined_array,return_index=True,return_counts=True)
duplicates = combined_array[ uniq_idx[counts>=2] ]  # <--- Get duplicates

top_5_indices = np.argsort(y_pred[0], axis=0)

top_5_numbers = []
for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)

print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)

print("The predicted numbers for the lottery game are:", temp)
print("The actual numbers in the last lottery game were:", prediction[0])
print("\n")
print("winners:", duplicates)
print(len(duplicates)/20)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 [==============================] - 4s 4s/step
The top 5 indices [ 0 18 17 19  2  5 16  1  4  3 15  6  7 13 10 11 12 14  9  8]
The top 5 numbers: [4, 64, 61, 68, 12, 23, 60, 11, 22, 20, 59, 31, 36, 54, 45, 49, 52, 58, 44, 41]
The predicted numbers for the lottery game are: [ 3 10 12 18 21 23 29 33 38 41 43 47 50 53 57 58 59 61 64 68]
The actual numbers in the last lottery game were: [ 4 11 12 20 22 23 31 36 41 44 45 49 52 54 58 59 60 61 64 68]


winners: [12 23 41 58 59 61 64 68]
0.4


# Predict the next set of numbers

In [127]:
next = df1.copy()
next = next.tail(window_length)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))
prediction = scaler.inverse_transform(y_next_pred).astype(int)
print("The predicted numbers for the lottery game:", temp)
print("\n")
top_5_indices = np.argsort(y_next_pred[0], axis=0)[-5:]
top_5_numbers = []
for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)


print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)


1/1 [==============================] - 2s 2s/step
The predicted numbers for the lottery game: [ 3 10 12 18 21 23 29 33 38 41 43 47 50 53 57 58 59 61 64 68]


The top 5 indices [1 2 3 9 4]
The top 5 numbers: [9, 13, 17, 37, 21]


In [128]:
next = df1.copy()
next = next.tail(window_length)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))

# Get the actual values of the top 10 predictions
prediction = scaler.inverse_transform(y_next_pred).astype(int)

# Get the indices of the top 10 predictions (change axis=0 if predictions are along a different axis)
top_10_indices = np.argsort(y_pred[0], axis=0)[-10:]
top_5_indices = np.argsort(y_pred[0], axis=0)[-5:]

top_10_numbers = []
top_5_numbers = []

# index [0,2,4] usually are correct
# Loop through each index in top_10_indices
for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_10_numbers.append(number)

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_5_numbers.append(number)

print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")

print("The top 10 indices", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("The predicted numbers:", prediction)


1/1 [==============================] - 0s 24ms/step
The top 5 indices [11 12 14  9  8]
The top 5 numbers: [40, 44, 50, 37, 31]


The top 10 indices [15  6  7 13 10 11 12 14  9  8]
The top 10 numbers: [55, 24, 27, 47, 39, 40, 44, 50, 37, 31]


The predicted numbers: [[ 1  9 13 17 21 21 24 27 31 37 39 40 44 47 50 55 59 61 63 68]]
